In [1]:
import pymongo
import pandas as pd

data_mongo= pymongo.MongoClient('localhost:27017')

In [2]:
db= data_mongo['test']
rows=db['datarows']

In [3]:
drow1= pd.json_normalize(list(rows.find()[:150000]),sep='_')
drow2= pd.json_normalize(list(rows.find()[150000:300000]),sep='_')
drow3= pd.json_normalize(list(rows.find()[300000:450000]),sep='_')
drow4= pd.json_normalize(list(rows.find()[450000:600000]),sep='_')
drow5= pd.json_normalize(list(rows.find()[600000:750000]),sep='_')
drow6= pd.json_normalize(list(rows.find()[750000:900000]),sep='_')
drow7= pd.json_normalize(list(rows.find()[900000:1050000]),sep='_')
drow8= pd.json_normalize(list(rows.find()[1050000:1200000]),sep='_')
drow9= pd.json_normalize(list(rows.find()[1200000:]),sep='_')

In [4]:
df_row=pd.concat([drow1,drow2,drow3,drow4,drow5,drow6,drow7,drow8,drow9],axis=0)
df_row.shape

(1296780, 22)

In [5]:
df_row._id=df_row._id.astype(str)

In [6]:
df_row.dataRowType.unique()

array(['BEACON', 'GPS', 'BATTERY', nan], dtype=object)

In [6]:
data_gps=df_row[df_row.dataRowType=='GPS']
data_beacon= df_row[df_row.dataRowType=='BEACON']
data_battery= df_row[df_row.dataRowType=='BATTERY']

In [7]:
df_gps=data_gps[['UUID','updatedAt','dataRowData_lat','dataRowData_lng','dataRowData_gpsAlt','dataRowData_gpsVel','dataRowData_gpsFixed']]

In [8]:
df_bate=data_battery[['UUID','updatedAt','dataRowData_timestamp','dataRowData_battery']]

In [9]:
df_beacon=data_beacon[['UUID','updatedAt','dataRowData_timestamp','dataRowData_mac','dataRowData_battery','dataRowData_temperature','dataRowData_rssi','dataRowData_accelerometer']]

In [10]:
df_bate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100500 entries, 6 to 96776
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   UUID                   100500 non-null  object        
 1   updatedAt              100500 non-null  datetime64[ns]
 2   dataRowData_timestamp  100500 non-null  object        
 3   dataRowData_battery    98833 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 3.8+ MB


In [11]:
df_gps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321019 entries, 1 to 96773
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   UUID                  321019 non-null  object        
 1   updatedAt             321019 non-null  datetime64[ns]
 2   dataRowData_lat       319620 non-null  float64       
 3   dataRowData_lng       319622 non-null  float64       
 4   dataRowData_gpsAlt    320871 non-null  float64       
 5   dataRowData_gpsVel    320871 non-null  float64       
 6   dataRowData_gpsFixed  320871 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 19.6+ MB


In [12]:
df_beacon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875260 entries, 0 to 96779
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   UUID                       875260 non-null  object        
 1   updatedAt                  875260 non-null  datetime64[ns]
 2   dataRowData_timestamp      875260 non-null  object        
 3   dataRowData_mac            875260 non-null  object        
 4   dataRowData_battery        875260 non-null  float64       
 5   dataRowData_temperature    875260 non-null  float64       
 6   dataRowData_rssi           875260 non-null  float64       
 7   dataRowData_accelerometer  230272 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 60.1+ MB


In [20]:
df_gps[df_gps.UUID=='0004A30B00F842B6']

,UUID,updatedAt,dataRowData_lat,dataRowData_lng,dataRowData_gpsAlt,dataRowData_gpsVel,dataRowData_gpsFixed
22667,0004A30B00F842B6,2023-03-10 21:39:56.083,-32.849794,-63.580536,0.0,0.055,1.0
22950,0004A30B00F842B6,2023-03-10 21:54:03.019,-32.849809,-63.580539,225.6,0.003,1.0
23195,0004A30B00F842B6,2023-03-10 22:06:59.113,-32.849797,-63.580911,224.8,0.201,1.0
23403,0004A30B00F842B6,2023-03-10 22:21:14.271,-32.849797,-63.580911,224.8,0.201,1.0
23648,0004A30B00F842B6,2023-03-10 23:15:22.910,NaN,NaN,0.0,0.000,1.0
...,...,...,...,...,...,...,...
96413,0004A30B00F842B6,2023-04-05 11:03:06.802,-34.205670,-64.031680,184.1,0.257,1.0
96431,0004A30B00F842B6,2023-04-05 11:16:03.558,-34.205652,-64.031674,169.5,0.095,1.0
96449,0004A30B00F842B6,2023-04-05 11:30:18.080,-34.205808,-64.031951,173.0,1.016,1.0
96482,0004A30B00F842B6,2023-04-05 11:47:38.280,-34.205672,-64.031708,0.0,0.022,1.0


In [13]:
vaca_1=df_gps[df_gps.UUID=='0004A30B00F8A181']
#vaca_1.updatedAt= pd.to_datetime(vaca_1.updatedAt)

In [17]:
import folium
datos_gps =vaca_1[['dataRowData_lat','dataRowData_lng']]

mapa = folium.Map(location=datos_gps.iloc[0].values,zoom_start=15,width="70%",height="60%")

for i , d in datos_gps.iterrows():
  folium.Marker(location=[d['dataRowData_lat'],d['dataRowData_lng']]).add_to(mapa)
  

folium.PolyLine(locations=datos_gps,color='blue').add_to(mapa)
mapa